# GridSearchCV

In [1]:
from sklearn import svm,datasets
iris = datasets.load_iris()

In [2]:
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [3]:
import pandas as pd
df = pd.DataFrame(iris['data'],columns=iris['feature_names'])
df['flower'] = iris['target']
df['flower'] = df['flower'].apply(lambda x: iris['target_names'][x])
df.sample(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
33,5.5,4.2,1.4,0.2,setosa
74,6.4,2.9,4.3,1.3,versicolor
34,4.9,3.1,1.5,0.2,setosa
140,6.7,3.1,5.6,2.4,virginica
134,6.1,2.6,5.6,1.4,virginica


In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(iris['data'],iris['target'],test_size=0.3)

In [5]:
model = svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.9777777777777777

### KFold cross_val_score

In [6]:
from sklearn.model_selection import cross_val_score

In [7]:
cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),iris['data'],iris['target'], cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [8]:
cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris['data'],iris['target'], cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [9]:
cross_val_score(svm.SVC(kernel='linear',C=20,gamma='auto'),iris['data'],iris['target'], cv=5)

array([1.        , 1.        , 0.9       , 0.93333333, 1.        ])

### Grid Search

In [10]:
# manually
import numpy as np
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris['data'], iris['target'], cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

In [11]:
# using gridsearch class
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma='auto'),{
    'C' : [1,10,20],
    'kernel' : ['rbf','linear']
},cv=5, return_train_score=False)

clf.fit(iris['data'],iris['target'])

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 10, 20], 'kernel': ['rbf', 'linear']})

In [12]:
clf.cv_results_

{'mean_fit_time': array([0.00200744, 0.00179553, 0.00100436, 0.00059218, 0.00058999,
        0.00119672]),
 'std_fit_time': array([0.00064699, 0.00074635, 0.00064144, 0.00048371, 0.00048198,
        0.00039847]),
 'mean_score_time': array([0.00078545, 0.00139613, 0.00058966, 0.0003984 , 0.00179586,
        0.00039897]),
 'std_score_time': array([0.00039313, 0.00101765, 0.00078355, 0.00048794, 0.00159509,
        0.00048864]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [13]:
df = pd.DataFrame(clf.cv_results_)
df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002007,0.000647,0.000785,0.000393,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001796,0.000746,0.001396,0.001018,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001004,0.000641,0.000590,0.000784,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000592,0.000484,0.000398,0.000488,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000590,0.000482,0.001796,0.001595,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5


In [14]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [15]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'multimetric_',
 'n_fe

In [16]:
clf.best_score_

0.9800000000000001

In [17]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

### RandomizedSearchCV

In [18]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'),{
        'C' : [1,10,20],
        'kernel' : ['rbf','linear']
    }, 
    cv=5, 
    return_train_score=False,
    n_iter=3
)

In [19]:
rs.fit(iris['data'],iris['target'])

RandomizedSearchCV(cv=5, estimator=SVC(gamma='auto'), n_iter=3,
                   param_distributions={'C': [1, 10, 20],
                                        'kernel': ['rbf', 'linear']})

In [20]:
df = pd.DataFrame(rs.cv_results_)
df.head()[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,20,rbf,0.966667
2,20,linear,0.966667
